In [0]:
%tensorflow_version 1.x

In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
# Cloning BERT git repo
!git clone https://github.com/google-research/bert.git
!mv bert bert-master

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 17.66 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [4]:
# Downloading BERT-Base, Cased Model
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip -O cased.zip

--2020-03-19 18:16:49--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.8.16, 2607:f8b0:4004:807::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.8.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404261442 (386M) [application/zip]
Saving to: ‘cased.zip’

cased.zip           100%[===================>] 385.53M   142MB/s    in 2.7s    

2020-03-19 18:16:52 (142 MB/s) - ‘cased.zip’ saved [404261442/404261442]



In [5]:
# Unzipping the downloaded folder and placing it into root/model
!mkdir model/
!unzip cased.zip -d model/
!rm cased.zip
!cd model/cased_L-12_H-768_A-12 && mv * .. && cd .. && rm -r cased_L-12_H-768_A-12

# Moving the model folder into root/bert-master
!mv model bert-master

Archive:  cased.zip
   creating: model/cased_L-12_H-768_A-12/
  inflating: model/cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: model/cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: model/cased_L-12_H-768_A-12/vocab.txt  
  inflating: model/cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: model/cased_L-12_H-768_A-12/bert_config.json  


In [7]:
## MAKE SURE YOUR CSV FILES TO THE ROOT THEN RUN THIS CELL!!!!!!!!!
!mkdir dataset
!mv imdb_train.csv dataset/
!mv imdb_test.csv dataset/

mkdir: cannot create directory ‘dataset’: File exists
mv: cannot stat 'imdb_train.csv': No such file or directory
mv: cannot stat 'imdb_test.csv': No such file or directory


In [8]:
# # If needed:
# !pip3 install pandas, sklearn

import pandas as pd
from sklearn.model_selection import train_test_split

# Read data from csv file
df_train = pd.read_csv('dataset/imdb_train.csv')
df_test = pd.read_csv('dataset/imdb_test.csv')

# Creating the unique identifiers for the dataset
ids = []
for i in range(len(df_train)):
  ids.append(i)
pd_ids = pd.DataFrame(ids)
df_train.insert(0, "id", pd_ids)
# Converting sentiments from "positive" and "negative" to 1 and 0
df_train['sentiment'] = df_train['sentiment'].map({'positive': 1, 'negative': 0})

# Repeating the above processes for the test set
idx = []
for i in range(len(df_test)):
  idx.append(i)
pd_idx = pd.DataFrame(idx)
df_test.insert(0, "id", pd_idx)
df_test['sentiment'] = df_test['sentiment'].map({'positive': 1, 'negative': 0})

# # Create new dataframes in the format required by BERT for train, dev data
df_bert = pd.DataFrame({'guid': df_train['id'],
                        'label': df_train['sentiment'],
                        'alpha': ['a'] * df_train.shape[0],
                        'text': df_train['review']})

# Split into test, dev
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

# Create new dataframe for test data
df_bert_test = pd.DataFrame({'guid': df_test['id'],
                             'text': df_test['review']})

# Output tsv file, no header for train and dev
!mkdir bert-master/dataset
df_bert_train.to_csv('bert-master/dataset/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('bert-master/dataset/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('bert-master/dataset/test.tsv', sep='\t', index=False, header=True)

print(df_bert_train.head())
print("-" * 100)
print(df_bert_test.head())

        guid  label alpha                                               text
35537  35537      0     a  I saw this director's "Woman On The Beach" and...
2305    2305      1     a  'Shock Corridor (1963)' was my first film from...
33760  33760      0     a  After Chaplin made one of his best films: Doug...
1437    1437      0     a  One of those classics, held up next to "Deep T...
33292  33292      1     a  The Lion King 1 1/2 is a very cute story to go...
----------------------------------------------------------------------------------------------------
   guid                                               text
0     0  One of the other reviewers has mentioned that ...
1     1  A wonderful little production. <br /><br />The...
2     2  I thought this was a wonderful way to spend ti...
3     3  Basically there's a family where a little boy ...
4     4  Petter Mattei's "Love in the Time of Money" is...


In [9]:
!nvidia-smi

Thu Mar 19 18:19:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [11]:
!cd bert-master && CUDA_VISIBLE_DEVICES=0 python3 run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=./dataset --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./model/bert_model.ckpt --max_seq_length=128 --train_batch_size=2 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=./bert_output/ --do_lower_case=False --save_checkpoints_steps 1000

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0319 18:22:28.860811 139641452205952 deprecation.py:323] From /tensorflow-1.15.0/python3.6/tensorflow_core/python/ops/math_grad.py:1375: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:training_loop marked as finished
I0319 18:22:33.524011 139641452205952 error_handling.py:101] training_loop marked as finished
Traceback (most recent call last):
  File "/tensorflow-1.15.0/python3.6/tensorflow_core/python/framework/ops.py", line 5628, in init_scope
    yield
  File "/tensorflow-1.15.0/python3.6/tensorflow_core/python/ops/variables.py", line 1818, in _init_from_args
    initial_value(), name="initial_value", dtype=dtype)
  File "/tensorflow-1.15.0/python3.6/tensorflow_core/python/ops/variable_scope.py", line 905, in <lambda>
    partiti